***DNA Assembly using Hamiltonian Path approach***
1. Read the shotgun reads from the fasta file.
2. Create a graph representation of the overlaps between reads.
3. Find the Hamiltonian path in the graph
4. Stitch the reads together based on the path to obtain the reconstructed genome sequence.

In [78]:
def parse_fasta(file_path):
    reads = []
    with open(file_path, "r") as file:
        lines = file.readlines()
        for line in lines:
            if line.startswith(">"):
                reads.append("")
            else:
                reads[-1] += line.strip()
    return reads


def build_graph(reads):
    graph = {}
    for i, read in enumerate(reads):
        graph[i] = []
        for j, other_read in enumerate(reads):
            if i != j:
                overlap = find_overlap(read, other_read)
                if overlap:
                    graph[i].append(j)
    return graph


def find_overlap(read1, read2):
    min_overlap = min(len(read1), len(read2)) // 2
    for i in range(min_overlap, 0, -1):
        if read1.endswith(read2[:i]):
            return i
    return 0


def find_hamiltonian_path(graph):
    stack = [0]
    path = []

    while stack:
        node = stack[-1]
        if node not in path:
            path.append(node)
        if len(path) == len(graph):
            return path
        found_neighbor = False
        for neighbor in graph[node]:
            if neighbor not in path:
                stack.append(neighbor)
                found_neighbor = True
                break
        if not found_neighbor:
            stack.pop()

    return None


def reconstruct_genome(reads, path):
    genome = reads[path[0]]
    for i in range(1, len(path)):
        overlap = find_overlap(reads[path[i - 1]], reads[path[i]])
        genome += reads[path[i]][overlap:]
    return genome[:1126]  # Take the first 1000 basepairs


# Parse the fasta file and extract the reads
reads = parse_fasta("reads.fasta")

# Build the graph representation
graph = build_graph(reads)

# Find the Hamiltonian path
hamiltonian_path = find_hamiltonian_path(graph)

if hamiltonian_path is not None:
    # Reconstruct the original genome sequence
    genome = reconstruct_genome(reads, hamiltonian_path)
    print("Reconstructed Genome Sequence:")
    print(genome)
    print("Length of Reconstructed Genome Sequence:")
    print(len(genome))
else:
    print("No Hamiltonian path found.")


Reconstructed Genome Sequence:
AGCCAATAGCAGATATGCCCATACCGCTGTATTCATAGCTTTCTCTACACGGCCTAAAAGCGGTCGACTGCACGGTCGACTGCACGCAGTCTTCCGGAGCCAATAGCAGATATGCCCATACTTATCACGTACGTAGATTCGAATACAAATTCGACAGATGTGGTTTAATGATTCCGCCTCCTATTACAACAGCCCCGAGGATCCTGCACTGAGTCTGAGGAGCTGGGGTGCAAATTAGACGGCCTAAAAGCGGTCGACTGCAAGGTAGGGGTACGTACATGTTTCCCATAGGCAGCGCCTGCCGCTGTTACAACAGCCGACATTGCGACACAATACCAGTTTTTATTGTGTCCATGTACCGCCTAACACTTATCACGTACGTACATGTTTCGGGAGAGAAAGGGGTGATGTTCTGTTATTAGACCGACGCCCCTAATTGGATCAATCAGGGTAGGTCATGGGAGGGGTGATGTTCGAATAAATGGCATATAAGCCCGGATCCGTCCTGTCTGCGACACTGACATGGATCCGTCCTGTCTGCGACGTTTCGGGTCGATAAAGCGTTGTCCGACGCCCCTAATTGGATCAATCCTGATGGTACTCCCCCTTCATTGCGCCCGTTTCCCATGTACCGCCTAGTACAAATTCGACAGATGTGGTTTATTCGATGGGTAGGTCATGGGAGTAGAGTCGGTGAGGAGCTGGGGTGAGTCTTCCTGATGGTACTCCCCCTGCACCATGAACGCGATTGCTAAACATGGATCCGTCCTGTCTGCGACATGGGAGGGGTGATGTTCTGTTATTGGTACTCCCCCTTCATTGGTCGGTAGAGTCGGTGAGGAGCTGGGGTGAGGATTAGCGCCCGTTTCCCATGTACCGCCTAACACTCGTGGTGAGCAGGAAATTATTCGCTTTACTAGTCACGTGCTCTAAAATAGCTTTCTCTACACGATCGAGTTGGGAGGATTA

**DNA Assembly using Eulerian Path approach**

1. Parse the fasta file and extract the shotgun reads.
2. Build a de Bruijn graph using the shotgun reads.
3. Find the Eulerian path in the de Bruijn graph.
4. Concatenate the nodes of the Eulerian path to obtain the reconstructed genome sequence.

The key part is choosing the appropriate value for k in the de Bruijn graph approach can be a challenging task, as it depends on various factors such as read length, sequencing errors, and genome complexity. 


In [79]:
def parse_fasta_file(file_path):
    sequences = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip()
            if line.startswith('>'):
                sequences.append('')
            else:
                sequences[-1] += line
    return sequences

def build_de_bruijn_graph(sequences, k):
    graph = {}
    for sequence in sequences:
        for i in range(len(sequence) - k + 1):
            kmer = sequence[i:i+k]
            prefix = kmer[:-1]
            suffix = kmer[1:]
            if prefix in graph:
                graph[prefix].append(suffix)
            else:
                graph[prefix] = [suffix]
    return graph

def find_eulerian_path(graph):
    start_node = list(graph.keys())[0]
    current_node = start_node
    path = [current_node]

    while True:
        if current_node not in graph:
            break
        next_node = graph[current_node].pop()
        if len(graph[current_node]) == 0:
            del graph[current_node]
        current_node = next_node
        path.append(current_node)

    return path

def reconstruct_genome(sequences, k):
    graph = build_de_bruijn_graph(sequences, k)
    eulerian_path = find_eulerian_path(graph)
    reconstructed_genome = eulerian_path[0]
    for node in eulerian_path[1:]:
        reconstructed_genome += node[-1]
    return reconstructed_genome[:1209]  # Take the first 1000 basepairs

# Provide the path to the fasta file
fasta_file = 'reads.fasta'

# Parse the fasta file and extract the shotgun reads
sequences = parse_fasta_file(fasta_file)

# Reconstruct the original genome sequence
reconstructed_sequence = reconstruct_genome(sequences, k=5)  # Adjust the value of k as needed
print("Reconstructed Genome Sequence using Eulerian Path approach:")
print(reconstructed_sequence)
print("Length of Reconstructed Genome Sequence:")
print(len(reconstructed_sequence))

Reconstructed Genome Sequence using Eulerian Path approach:
AGCCCATTACGTGATTCCGTGCTCTACCAGTTTCGGGAGAGATGGTCGGTGAAGTTTCATTGGTACTCCCCTTCATCTAAAACCGATCTGTTATTAGACCCCCGGCCCGTACATGTTCTGCCGTCCTGTCTGCCGCCGCTGCGAATGGATCCGGAGATGGTTGGGAGTGGAGAAAGGCATATAAGCCCGGAATACCCATAGGCCAATCCGGAGTGGAGGATTACTAGTCACGTACGAAAGCCAATCCGGCCAATAAGCGCCTGCGAATCCGCCTCCTAGACATGGCAGCGCAAGGCTTTACAACAGTGGGAGTGGGTACATGGTTGGGTGATTCGGGAGGGGTAGAGCCGACAAAACAGCTTCAATCCGGAGAGATATAAAACAGTTATTGTGTCCCATGTTTTATCTGCGACACGTGCTCTAAATGGCGCCCGTTTCCGGAGTGGAGTGATGTATAAGCCCGTGCTCTAAATGGTTATCTGCGAATCCGGAGATTAGCGTTTCGAATAGCAGATAAAGGTAGGTCGGTGAATCCGGAGTTAATGGGGTATTATGCCGTCCTGTACGTGCTCTACACAAGGTACTAGTCCGGCCCATTGGTCACGTACCGATATGATTTGCAAGGCATATAGAGATGTTTCTCTCGTTGGTTATCTGCGACATTGGGTAGGGGTATAGCTTTACTAGTCATTGCGACACTGAATAAAAACCGCCCGTAGAGGCCCGGAGCTTCAGCTGCGAATGGTCGGGTCGATCCGTTTCGGGTCGGTGCAAGGCATATAGACATAGGATCATGTGGTTTCGGTGATCGATTAGAGGCTTTACTCCCCTTCAGCTTTACTAGTCACGTGAGCTGGGAGTGGGTGATGGCAGAGGATCTACCAGTCATAGACAACAGCCGCCGACGTGTCTGTCCCATTAGCGCCTATTCATTGCGACA